In [1]:
import numpy as np
#from urllib import urlopen
import urllib
import scipy.optimize
import random
from math import exp
from math import log

def parseData(fname):
  for l in urllib.urlopen(fname):
    yield eval(l)

print("Reading data...")
data = list(parseData("beer_50000.json"))
print("done")

def feature(datum):
  feat = [1, datum['review/taste'], datum['review/appearance'], datum['review/aroma'], datum['review/palate'], datum['review/overall']]
  return feat

#X = [feature(d) for d in data]
#y = [d['beer/ABV'] >= 6.5 for d in data]

def inner(x,y):
  return sum([x[i]*y[i] for i in range(len(x))])

def sigmoid(x):
  return 1.0 / (1 + exp(-x))

##################################################
# Logistic regression by gradient ascent         #
##################################################

# NEGATIVE Log-likelihood
def f(theta, X, y, lam):
  loglikelihood = 0
  for i in range(len(X)):
    logit = inner(X[i], theta)
    loglikelihood -= log(1 + exp(-logit))
    if not y[i]:
      loglikelihood -= logit
  for k in range(len(theta)):
    loglikelihood -= lam * theta[k]*theta[k]
  # for debugging
  # print("ll =" + str(loglikelihood))
  return -loglikelihood

# NEGATIVE Derivative of log-likelihood
def fprime(theta, X, y, lam):
  dl = [0]*len(theta)
  for i in range(len(X)):
    logit = inner(X[i], theta)
    for k in range(len(theta)):
      dl[k] += X[i][k] * (1 - sigmoid(logit))
      if not y[i]:
        dl[k] -= X[i][k]
  for k in range(len(theta)):
    dl[k] -= lam*2*theta[k]
  return numpy.array([-x for x in dl])

#X_train = X
#y_train = y

##################################################
# Train                                          #
##################################################

def train(lam,X,X_train,y_train):
  theta,_,_ = scipy.optimize.fmin_l_bfgs_b(f, [0]*len(X[0]), fprime, pgtol = 10, args = (X_train, y_train, lam))
  return theta

##################################################
# Predict                                        #
##################################################

def performance(theta,X,y_train):
  scores = [inner(theta,x) for x in X]
  predictions = [s > 0 for s in scores]
  correct = [(a==b) for (a,b) in zip(predictions,y_train)]
  acc = sum(correct) * 1.0 / len(correct)
  return acc

##################################################
# Validation pipeline                            #
##################################################

#lam = 1.0

#theta = train(lam)
#acc = performance(theta)
#print("lambda = " + str(lam) + ":\taccuracy=" + str(acc))


Reading data...
done


In [2]:
print "Question 1:"
X_train = [feature(d) for d in data[:len(data)/3] ]
X_valid = [feature(d) for d in data[len(data)/3:(len(data)*2)/3] ]
X_test = [feature(d) for d in data[(len(data)*2)/3:] ]
y_train = [d['beer/ABV'] >= 6.5 for d in data[:len(data)/3]]
y_valid = [d['beer/ABV'] >= 6.5 for d in data[len(data)/3:(len(data)*2)/3]]
y_test = [d['beer/ABV'] >= 6.5 for d in data[(len(data)*2)/3:]]
lam = 1.0
theta = train(lam,X_train,X_train,y_train)
acc = performance(theta,X_valid,y_valid)
print "valid acc: ", acc
acc = performance(theta,X_test,y_test)
print "test acc: ", acc


Question 1:


NameError: global name 'numpy' is not defined

In [ ]:
print "Question 2:" 
print 'feature function'
print ''

import re

def wc(key,rv):
    tt = re.compile('\w+').findall(rv.lower())
    return tt.count(key)

def Q2feature(datum):
    rv = datum['review/text']
    return [1,wc('lactic',rv),wc('tart',rv),
    wc('sour',rv),wc('citric',rv),wc('sweet',rv),
    wc('acid',rv),wc('hop',rv),wc('fruit',rv),wc('salt',rv),wc('spicy',rv)]
    

In [ ]:
print 'Question 3:'
FN = float(0)
TN = float(0)
TP = float(0)
FP = float(0)
X_train = [Q2feature(d) for d in data[:len(data)/3] ]
y_train = [d['beer/ABV'] >= 6.5 for d in data[:len(data)/3]]
X_test = [Q2feature(d) for d in data[(len(data)*2)/3:] ]
y_test = [d['beer/ABV'] >= 6.5 for d in data[(len(data)*2)/3:]]
lam = 1.0
theta = train(lam,X_train,X_train,y_train)
scores = [inner(theta,x) for x in X_test]
predictions = [s > 0 for s in scores]
for (p,r) in zip(predictions,y_test):
    if p and r:
        TP +=1 
    elif p == 1 and r == 0:
        FP +=1
    elif p == 0 and r==1:
        FN += 1
    elif p == 0 and r == 0:
        TN+=1
BER =  0.5*(FP/(TN+FP) + FN/(FN+TP))
print 'TN:',TN
print 'TP:',TP
print 'FN:',FN
print 'FP:',FP
print 'balanced error rate: ', BER
print ''

In [ ]:
print 'Question 4:'
print 'Used features in Question2'
lams = [0,0.01,0.1,1,100]
X_train  = [Q2feature(d) for d in data[:len(data)/3]]
y_train = [d['beer/ABV'] >= 6.5 for d in data[:len(data)/3]]
X_valid = [Q2feature(d) for d in data[len(data)/3:(len(data)*2)/3]]
y_valid = [d['beer/ABV'] >= 6.5 for d in data[len(data)/3:(len(data)*2)/3]]
X_test = [Q2feature(d) for d in data[(len(data)*2)/3:]]
y_test = [d['beer/ABV'] >= 6.5 for d in data[(len(data)*2)/3:]]
bestacc = 0
lam = None
for lam in lams:
    print 'lam = ', lam
    theta = train(lam,X_train,X_train,y_train)
    tracc = performance(theta,X_train,y_train)
    print 'acc on train:', tracc
    vacc = performance(theta,X_valid,y_valid)
    print 'acc on valid:', vacc
    teacc = performance(theta,X_test,y_test)
    print 'acc on test:', teacc
    print '----------------------------------------------------'
    if bestacc < vacc:
        bestacc = vacc    
        bestlam = lam
print 'best lamda: ', bestlam
acc = performance(theta,X_train,y_train)
print 'acc on train:', acc
print 'acc on valid:', bestacc
acc = performance(theta,X_test,y_test)
print 'acc on test:', acc

In [ ]:
from sklearn.decomposition import PCA
from collections import defaultdict

print "Question 5:"
X_train  = [Q2feature(d)[1:] for d in data[:len(data)/3]]
y_train = [d['beer/ABV'] >= 6.5 for d in data[:len(data)/3]]
X_valid = [Q2feature(d)[1:] for d in data[len(data)/3:(len(data)*2)/3]]
y_valid = [d['beer/ABV'] >= 6.5 for d in data[len(data)/3:(len(data)*2)/3]]
X_test = [Q2feature(d)[1:] for d in data[(len(data)*2)/3:]]
y_test = [d['beer/ABV'] >= 6.5 for d in data[(len(data)*2)/3:]]
pca = PCA(n_components=10)
pca.fit(X_train)
print 'PCA components on train set:'
print pca.components_



In [ ]:
print 'Question 6:'
print np.matmul(X_train[0],(pca.components_).T)


In [ ]:
print 'Question 7:'
pcavar = pca.explained_variance_
print "Reconstruction error of d = 2:"
print 'covariance Matrix eig val: ', sum(pcavar[2:])*len(X_train)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

print 'Question 8:'
y = np.matmul(X_train,pca.components_[:2].T)
xipa = list()
yipa = list()
xoth = list()
yoth = list()
for i in range(len(y)):
    if data[i]['beer/style'] == 'American IPA':
        xipa.append(y[i][0])
        yipa.append(y[i][1])
    else:
        xoth.append(y[i][0])
        yoth.append(y[i][1])
plt.scatter(xoth,yoth,color= 'blue')        
plt.scatter(xipa,yipa,color= 'red')
#plt.scatter(xipa,yipa,color= 'red');
plt.title('first two features (IPA = red, others = blue)')
plt.xlabel('feature1')
plt.ylabel('feature2')
plt.show()
